### Imports

In [ ]:
import pandas as pd
import numpy as np

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA

from sklearn.cluster import KMeans

from feature_engineering_functions import ColumnsCombinator

from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram

### Create pipelines for different experiments

In [ ]:
from sklearn.cluster import KMeans


saldo_creator = ColumnsCombinator(arguments=["exports", "imports"],
                                  operators=["-"],
                                  new_column_name="saldo")
total_health_exp_est_creator= ColumnsCombinator(arguments=["health", "gdpp", "income"],
                                                operators=["*", "+"],
                                                new_column_name="total_health_exp")
# preprocessors
scaler = StandardScaler()
normalizer = Normalizer()
pca = PCA(n_components=4)

# models
kmeans = KMeans(n_clusters=3, random_state=0)

